# 1. Document Classification

In [2]:
import tensorflow as tf
print(tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
for device in gpus:
    tf.config.experimental.set_memory_growth(device, True)

2.4.1


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
import csv
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## (a) Preprocess

In [4]:
def import_pretrained_embeddings(path):
    embeddings_dict = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

In [5]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

In [6]:
author_encode = {'fd':0, 'ja': 1, 'acd': 2}

In [7]:
def import_data(filepaths_dict, embedding):
    
    stop_words = set(nltk.corpus.stopwords.words('english'))
    
#     lst_all = []
    x_all = []
    y_all = []
    
    for author, filepath in filepaths_dict.items():
        # get rows from .txt file
        with open(filepath) as f:
            reader = f.read()
            reader = reader.split("\n\n")

            tokenized_paragraphs = []
            print(f"author {author} has {len(reader)} samples")
            # iterate through every text row to clean it up
            for sample_idx, paragraph in enumerate(reader):
                # 1. remove punctuations
                paragraph = paragraph.translate(str.maketrans('','',string.punctuation))
                paragraph = paragraph.replace('\n', ' ')
                # 2. tokenize
                tokens = nltk.word_tokenize(paragraph)
                # 3. remove stop words

                vectors = []
                
                for token in tokens:
                    if not token in stop_words:
                        try:
                            vector = embedding[token.lower()]
                            vectors.append(vector)
                        except KeyError:
                            continue
                            
#                 thisrow = [vectors, author]
#                 lst_all.append(thisrow)
                x_all.append(vectors)
                y_all.append(author_encode[author])
                
    
    return x_all, y_all



Run the functions defined above

In [8]:
embeddings_dict = import_pretrained_embeddings('glove.6B.50d.txt')

In [9]:
filepaths_dict = {'fd': './a4-data/q1/fd.txt',
            'acd': './a4-data/q1/acd.txt',
            'ja': './a4-data/q1/ja.txt'}

x, y = import_data(filepaths_dict, embeddings_dict)

author fd has 6055 samples
author acd has 2548 samples
author ja has 11495 samples


In [10]:
y = np.array(y)
y = tf.keras.utils.to_categorical(y, num_classes=3)

In [11]:
assert(len(x) == len(y) == 6055+2548+11495)

In [12]:
vocab_size = len(embeddings_dict.keys())

In [13]:
x = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=500)


### Split into training and testing

In [14]:
X_train, x, y_train, y = train_test_split(x, y, test_size=0.4, random_state=42)

In [15]:
X_val, X_test, y_val, y_test = train_test_split(x, y, test_size=0.25, random_state=28)

In [ ]:
print("Sizes of sets:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

There is an obscure bug with the Tensorflow + CuDNN install for GPUs that occurs when there is a remainder when creating batches of data. To avoid this error, I adjusted the training dat

In [18]:
X_train = X_train[0:12000]
y_train = y_train[0:12000]
X_val = X_val[0:6000]
y_val = y_val[0:6000]
X_test = X_test[0:2000]
y_test = y_test[0:2000]

## (b) Build your models

### LSTM using only last hidden state

In [18]:
# define model
last_hs_model = tf.keras.Sequential()
# last_hs_model.add(tf.keras.layers.Embedding(vocab_size, 50, input_length=500))
# model.add(tf.keras.layers.LSTM(100, return_sequences=True))
last_hs_model.add(tf.keras.layers.LSTM(100, batch_input_shape=(50,500,50)))
last_hs_model.add(tf.keras.layers.Dense(100, activation='relu'))
last_hs_model.add(tf.keras.layers.Dense(3, activation='softmax'))
print(last_hs_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (50, 100)                 60400     
_________________________________________________________________
dense (Dense)                (50, 100)                 10100     
_________________________________________________________________
dense_1 (Dense)              (50, 3)                   303       
Total params: 70,803
Trainable params: 70,803
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
last_hs_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
last_hs_history = last_hs_model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=50, epochs=12)

Epoch 1/4
240/240 [==============================] - 11s 27ms/step - loss: 0.7687 - accuracy: 0.6764 - val_loss: 0.5375 - val_accuracy: 0.7892
Epoch 2/4
240/240 [==============================] - 6s 25ms/step - loss: 0.5133 - accuracy: 0.7964 - val_loss: 0.4945 - val_accuracy: 0.8023
Epoch 3/4
240/240 [==============================] - 6s 25ms/step - loss: 0.4343 - accuracy: 0.8235 - val_loss: 0.4104 - val_accuracy: 0.8343
Epoch 4/4
240/240 [==============================] - 6s 25ms/step - loss: 0.3791 - accuracy: 0.8419 - val_loss: 0.4175 - val_accuracy: 0.8262


### LSTM using the element wise average of all the hidden states

In [19]:
# define model
all_hs_model = tf.keras.Sequential()
# all_hs_model.add(tf.keras.layers.Embedding(vocab_size, 50))
all_hs_model.add(tf.keras.layers.LSTM(100, return_sequences=True, batch_input_shape=(50,500,50)))
all_hs_model.add(tf.keras.layers.GlobalAveragePooling1D())
all_hs_model.add(tf.keras.layers.Dense(100, activation='relu'))
all_hs_model.add(tf.keras.layers.Dense(3, activation='softmax'))
print(all_hs_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (50, 500, 100)            60400     
_________________________________________________________________
global_average_pooling1d (Gl (50, 100)                 0         
_________________________________________________________________
dense (Dense)                (50, 100)                 10100     
_________________________________________________________________
dense_1 (Dense)              (50, 3)                   303       
Total params: 70,803
Trainable params: 70,803
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
all_hs_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
all_hs_history = all_hs_model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=50, epochs=12)

Epoch 1/8
240/240 [==============================] - 7s 27ms/step - loss: 0.4118 - accuracy: 0.8399 - val_loss: 0.4254 - val_accuracy: 0.8280
Epoch 2/8
240/240 [==============================] - 6s 25ms/step - loss: 0.3875 - accuracy: 0.8474 - val_loss: 0.4003 - val_accuracy: 0.8405
Epoch 3/8
240/240 [==============================] - 6s 26ms/step - loss: 0.3728 - accuracy: 0.8493 - val_loss: 0.3891 - val_accuracy: 0.8433
Epoch 4/8
240/240 [==============================] - 6s 25ms/step - loss: 0.3513 - accuracy: 0.8575 - val_loss: 0.3841 - val_accuracy: 0.8465
Epoch 5/8
240/240 [==============================] - 6s 25ms/step - loss: 0.3390 - accuracy: 0.8633 - val_loss: 0.3807 - val_accuracy: 0.8492
Epoch 6/8
240/240 [==============================] - 6s 25ms/step - loss: 0.3263 - accuracy: 0.8669 - val_loss: 0.3717 - val_accuracy: 0.8515
Epoch 7/8
240/240 [==============================] - 6s 25ms/step - loss: 0.3140 - accuracy: 0.8708 - val_loss: 0.3730 - val_accuracy: 0.8528
Epoch 

In [20]:
class_weights = {0: 0.6985, 1:0.8732, 2:0.4280}